<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_assisted_labeling/image_mal.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_assisted_labeling/image_mal.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Annotation Import
* This notebook will provide examples of each supported annotation type for image assets. It will cover the following:
    * Model-Assisted Labeling (MAL) - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
    * Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.

* For information on what types of annotations are supported per data type, refer to this documentation:
    * https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended

* Notes:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

# Installs

In [1]:
!pip install -q 'labelbox[data]'

     |████████████████████████████████| 172 kB 9.3 MB/s 
     |████████████████████████████████| 6.3 MB 15.2 MB/s 


# Imports

In [2]:
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.data.serialization import NDJsonConverter
from labelbox.schema.media_type import MediaType
import uuid
import json
import numpy as np

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [30]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project setup

We will be creating two projects, one for model-assisted labeling, and one for label imports

First, we create an ontology with all the possible tools and classifications supported for images. The official list of supported annotations to import can be found here:
- [Model-Assisted Labeling](https://docs.labelbox.com/docs/model-assisted-labeling) (annotations/labels are not submitted)
- [Ground Truth](https://docs.labelbox.com/docs/import-ground-truth) (annotations/labels are submitted)

In [31]:
ontology_builder = OntologyBuilder(
  tools=[ # List of Tool objects
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX, 
      name="box"), 
    Tool( # Polyline tool given the name "line"
      tool=Tool.Type.LINE, 
      name="line"), 
    Tool( # Point tool given the name "point"
      tool=Tool.Type.POINT, 
      name="point"), 
    Tool( # Polygon tool given the name "polygon"
      tool=Tool.Type.POLYGON, 
      name="polygon"), 
    Tool( # Segmentation mask tool given the name "mask"
      tool=Tool.Type.SEGMENTATION, 
      name="mask")], 
  classifications=[ # List of Classification objects
    Classification( # Text classification given the name "text"
      class_type=Classification.Type.TEXT,
      instructions="text"), 
    Classification( # Checklist classification given the name "text" with two options: "first_checklist_answer" and "second_checklist_answer"
      class_type=Classification.Type.CHECKLIST, 
      instructions="checklist", 
      options=[
        Option(value="first_checklist_answer"),
        Option(value="second_checklist_answer")            
      ]
    ), 
    Classification( # Radio classification given the name "text" with two options: "first_radio_answer" and "second_radio_answer"
      class_type=Classification.Type.RADIO, 
      instructions="radio", 
      options=[
        Option(value="first_radio_answer"),
        Option(value="second_radio_answer")
      ]
    )
  ]
)

To show the two different ways to upload annotations, we create two projects - one to showcase MAL (Model-Assisted Labeling) and one to showcase Label Import.

In [32]:
# Create two Labelbox projects
mal_project = client.create_project(
    name="image_mal_project",
    media_type=MediaType.Image
)
li_project = client.create_project(
    name="image_label_import_project",
    media_type=MediaType.Image
)

# Create one Labelbox dataset
dataset = client.create_dataset(name="image_annotation_import_demo_dataset")

# Grab an example image and create a Labelbox data row
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data=test_img_url)

# Setup your ontology / labeling editor
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor")) # Unless using a custom editor, do not modify this

mal_project.setup(editor, ontology_builder.asdict()) # Connect your ontology and editor to your MAL project
mal_project.datasets.connect(dataset) # Connect your dataset to your MAL project

li_project.setup(editor, ontology_builder.asdict()) # Connect your ontology and editor to your Label Import project
li_project.datasets.connect(dataset) # Connect your dataset to your Label Import project

### Create Label using Annotation Type Objects
* It is recommended to use the Python SDK's annotation types for importing into Labelbox.

### Object Annotations

In [33]:
point_annotation=ObjectAnnotation(
    value=Point(x=882,y=159), # Coordinates for this point annotation
    name="point" # Name of the tool in your ontology
)

box_annotation=ObjectAnnotation(
    value=Rectangle( # Coordinates for the top-left and bottom-right points of your bounding box, respectively
        start=Point(x=557,y=898),
        end=Point(x=852,y=1140)
    ),
    name="box" # Name of the tool in your ontology
)

polyline_annotation=ObjectAnnotation(
    value=Line( # Coordinates for the keypoints in your polyline
        points=[Point(x=2534.353, y=249.471),Point(x=2429.492, y=182.092),Point(x=2294.322, y=221.962),Point(x=2224.491, y=180.463),Point(x=2136.123, y=204.716),
                Point(x=1712.247, y=173.949),Point(x=1703.838, y=84.438),Point(x=1579.772, y=82.61),Point(x=1583.442, y=167.552),
                Point(x=1478.869, y=164.903),Point(x=1418.941, y=318.149),Point(x=1243.128, y=400.815),Point(x=1022.067, y=319.007),
                Point(x=892.367, y=379.216),Point(x=670.273, y=364.408),Point(x=613.114, y=288.16),Point(x=377.559, y=238.251),
                Point(x=368.087, y=185.064),Point(x=246.557, y=167.286),Point(x=236.648, y=285.61),Point(x=90.929, y=326.412)]
    ),
    name="line" # Name of the tool in your ontology
)

polygon_annotation=ObjectAnnotation(
    value=Polygon( # Coordinates for the verticies of your polygon
        points=[Point(x=1489.581,y=183.934),Point(x=2278.306,y=256.885),Point(x=2428.197,y=200.437),Point(x=2560.0,y=335.419),
                Point(x=2557.386,y=503.165),Point(x=2320.596,y=503.103),Point(x=2156.083, y=628.943),Point(x=2161.111,y=785.519),
                Point(x=2002.115, y=894.647),Point(x=1838.456,y=877.874),Point(x=1436.53,y=874.636),Point(x=1411.403,y=758.579),
                Point(x=1353.853,y=751.74),Point(x=1345.264, y=453.461),Point(x=1426.011,y=421.129)]
    ),
    name="polygon" # Name of the tool in your ontology
)

mask_annotation=ObjectAnnotation(
    value=Mask( # Numpy array representation of a segmentation mask with the corresponding values within the array that represent the segmentation mask
        mask=MaskData(
            arr=np.zeros([400,450,3],dtype='uint8') # Creating an example numpy array to represent a mask creating a square from pixels 0,0 to 128,128
        ),
        color=(0,0,0) # Identifying what values in the numpy array correspond to the mask annotation (since I made a 3-D array with all zeroes, here it's 0,0,0)
    ),
    name="mask" # Name of the tool in your ontology
)

### Classification Annotations

In [34]:
text_annotation=ClassificationAnnotation(
    value=Text( # String value for the text annotation
        answer="the answer to the text question" 
    ), 
    name="text" # Name of the classification in your ontology
)

checklist_annotation=ClassificationAnnotation(
    value=Checklist(
        answer=[ # List of the checklist answers in your ontology
            ClassificationAnswer(name="first_checklist_answer"),
            ClassificationAnswer(name="second_checklist_answer")
        ]
    ), 
    name="checklist" # Name of the classification in your ontology
)

radio_annotation=ClassificationAnnotation(
    value=Radio(
        answer=ClassificationAnswer(
            name="second_radio_answer" # Name of the radio answer in your ontology
        )
    ), 
    name="radio" # Name of the classification in your ontology
)

### Create a Label object with all of our annotations

In [35]:
# Create a Label object by identifying the applicavle data row in Labelbox and providing a list of annotations
label = Label(
    data=ImageData(
        uid=data_row.uid),
    annotations = [
        point_annotation, box_annotation, polyline_annotation, polygon_annotation, mask_annotation,
        text_annotation, checklist_annotation, radio_annotation
    ]
)

# Create urls to mask data for upload
def signing_function(obj_bytes: bytes) -> str:
    url = client.upload_data(content=obj_bytes, sign=True)
    return url

label.add_url_to_masks(signing_function)

label.__dict__

/usr/local/lib/python3.7/dist-packages/labelbox/data/annotation_types/classification/classification.py:85: UserWarning: Dropdown classification is deprecated and will be removed in a future release
  warnings.warn("Dropdown classification is deprecated and will be "


{'annotations': [ObjectAnnotation(name='point', feature_schema_id=None, extra={}, value=Point(extra={}, x=882.0, y=159.0), classifications=[]),
  ObjectAnnotation(name='box', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=557.0, y=898.0), end=Point(extra={}, x=852.0, y=1140.0)), classifications=[]),
  ObjectAnnotation(name='line', feature_schema_id=None, extra={}, value=Line(extra={}, points=[Point(extra={}, x=2534.353, y=249.471), Point(extra={}, x=2429.492, y=182.092), Point(extra={}, x=2294.322, y=221.962), Point(extra={}, x=2224.491, y=180.463), Point(extra={}, x=2136.123, y=204.716), Point(extra={}, x=1712.247, y=173.949), Point(extra={}, x=1703.838, y=84.438), Point(extra={}, x=1579.772, y=82.61), Point(extra={}, x=1583.442, y=167.552), Point(extra={}, x=1478.869, y=164.903), Point(extra={}, x=1418.941, y=318.149), Point(extra={}, x=1243.128, y=400.815), Point(extra={}, x=1022.067, y=319.007), Point(extra={}, x=892.367, y=379.216), Point(extra

### Model Assisted Labeling 

To do model-assisted labeling, we need to convert a Label object into an NDJSON. 

This is easily done with using the NDJSONConverter class

We will create a Label called mal_label which has the same original structure as the label above

Notes:
* the NDJsonConverter takes in a list of labels

In [16]:
import copy # We import this python package so we can create copies of our hard-coded annotations and upload one copy to each

In [39]:
# Make a copy of the label to upload to the MAL project
mal_label = copy.deepcopy(label)

# Convert our label from a Labelbox class object to the underlying NDJSON format required for upload - uploads can be directly built in this syntax as well
mal_ndjson = list(NDJsonConverter.serialize([mal_label]))
mal_ndjson

[{'classifications': [],
  'dataRow': {'id': 'cl63z7sm83xv608vs0bpjb51e'},
  'name': 'point',
  'point': {'x': 882.0, 'y': 159.0},
  'uuid': 'adbdf3bf-6474-4beb-8726-3ee4bb3a35af'},
 {'bbox': {'height': 242.0, 'left': 557.0, 'top': 898.0, 'width': 295.0},
  'classifications': [],
  'dataRow': {'id': 'cl63z7sm83xv608vs0bpjb51e'},
  'name': 'box',
  'uuid': '29503cec-4a69-43d3-8ab1-64d6813d93d4'},
 {'classifications': [],
  'dataRow': {'id': 'cl63z7sm83xv608vs0bpjb51e'},
  'line': [{'x': 2534.353, 'y': 249.471},
   {'x': 2429.492, 'y': 182.092},
   {'x': 2294.322, 'y': 221.962},
   {'x': 2224.491, 'y': 180.463},
   {'x': 2136.123, 'y': 204.716},
   {'x': 1712.247, 'y': 173.949},
   {'x': 1703.838, 'y': 84.438},
   {'x': 1579.772, 'y': 82.61},
   {'x': 1583.442, 'y': 167.552},
   {'x': 1478.869, 'y': 164.903},
   {'x': 1418.941, 'y': 318.149},
   {'x': 1243.128, 'y': 400.815},
   {'x': 1022.067, 'y': 319.007},
   {'x': 892.367, 'y': 379.216},
   {'x': 670.273, 'y': 364.408},
   {'x': 613.

In [18]:
# Upload our label using Model-Assisted Labeling
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = mal_project.uid, 
    name="mal_job", 
    predictions=mal_ndjson)

In [19]:
# Errors will appear for each annotation that failed.
# Empty list means that there were no errors
# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
print("Errors:", upload_job.errors)

Errors: []


### Label Import

Label import is very similar to model-assisted labeling. We will create a Label called li_label which has the same original structure as the label above

In [36]:
# Make a copy of the label to upload to the Label Import project
li_label = copy.deepcopy(label)

# Convert our label from a Labelbox class object to the underlying NDJSON format required for upload - uploads can be directly built in this syntax as well
li_ndjson = list(NDJsonConverter.serialize([li_label]))
li_ndjson

[{'classifications': [],
  'dataRow': {'id': 'cl63z7sm83xv608vs0bpjb51e'},
  'name': 'point',
  'point': {'x': 882.0, 'y': 159.0},
  'uuid': '078ba35d-733e-403f-9299-519685ef2998'},
 {'bbox': {'height': 242.0, 'left': 557.0, 'top': 898.0, 'width': 295.0},
  'classifications': [],
  'dataRow': {'id': 'cl63z7sm83xv608vs0bpjb51e'},
  'name': 'box',
  'uuid': '1a4e8c0e-98e5-4adc-9723-552067e09d1c'},
 {'classifications': [],
  'dataRow': {'id': 'cl63z7sm83xv608vs0bpjb51e'},
  'line': [{'x': 2534.353, 'y': 249.471},
   {'x': 2429.492, 'y': 182.092},
   {'x': 2294.322, 'y': 221.962},
   {'x': 2224.491, 'y': 180.463},
   {'x': 2136.123, 'y': 204.716},
   {'x': 1712.247, 'y': 173.949},
   {'x': 1703.838, 'y': 84.438},
   {'x': 1579.772, 'y': 82.61},
   {'x': 1583.442, 'y': 167.552},
   {'x': 1478.869, 'y': 164.903},
   {'x': 1418.941, 'y': 318.149},
   {'x': 1243.128, 'y': 400.815},
   {'x': 1022.067, 'y': 319.007},
   {'x': 892.367, 'y': 379.216},
   {'x': 670.273, 'y': 364.408},
   {'x': 613.

In [37]:
# Upload our label using Label Import
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = li_project.uid, 
    name="label_import_job", 
    labels=li_ndjson)

In [38]:
# Errors will appear for each annotation that failed.
# Empty list means that there were no errors
# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
print("Errors:", upload_job.errors)

Errors: []
